In [3]:
# 準備數據集
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from models.model_Tudui import *
import time

train_data = torchvision.datasets.CIFAR10(root='./dataset/CIFAR10', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root='./dataset/CIFAR10', train=False, transform=torchvision.transforms.ToTensor(), download=True)


train_data_size = len(train_data)
test_data_size = len(test_data)

print(f'訓練數據集的長度為: {train_data_size}')
print(f'測試數據集的長度為: {test_data_size}')

# 利用 dataloader 加載數據集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 建立網路模型
tudui = Tudui()
if torch.cuda.is_available():
    tudui = tudui.cuda()

# 損失函數
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# 優化器
# learning_rate = 0.01
# 1e-2
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)




# 設至訓練網路的一些參數
# 紀錄訓練的次數
total_train_step = 0
# 紀錄測試的次數
total_test_step = 0
# 訓練的輪數
epoch = 20


# 添加 tensorboard
writer = SummaryWriter("logs_train")

start_time = time.time()
for i in range(epoch):
    print(f'---------第 {i+1} 輪訓練開始---------')
    # 訓練步驟開始
    tudui.train()
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)

        # 優化器優化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # loss.item() 轉成 python 物件
        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print(f'訓練次數: {total_train_step}, Loss: {round(loss.item(),4)}')
            writer.add_scalar('train_loss', loss.item(), total_train_step)
    end_time = time.time()
    print(f'訓練花費時間:{round(end_time-start_time, 4)}秒') 
    # 測試步驟開始
    tudui.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
    accuracy_rate = total_accuracy.item()/test_data_size
    print(f'整體測試集上的loss: {total_test_loss}')
    print(f'整體測試集上的正確率: {accuracy_rate}')
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    writer.add_scalar('test_accuracy', accuracy_rate, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(tudui, f"checkpoints/tudui_{i}.pth")
    # torch.save(tudui.state_dict(), f"checkpoints/tudui_{i}.pth")
    print('模型已保存')


writer.close()

Files already downloaded and verified
Files already downloaded and verified
訓練數據集的長度為: 50000
測試數據集的長度為: 10000
---------第 1 輪訓練開始---------
訓練次數: 100, Loss: 2.2888
訓練次數: 200, Loss: 2.2856
訓練次數: 300, Loss: 2.2597
訓練次數: 400, Loss: 2.18
訓練次數: 500, Loss: 2.0622
訓練次數: 600, Loss: 1.9869
訓練次數: 700, Loss: 2.0323
訓練花費時間:7.0595
整體測試集上的loss: 319.4511944055557
整體測試集上的正確率: 0.2761
模型已保存
---------第 2 輪訓練開始---------
訓練次數: 800, Loss: 1.8688
訓練次數: 900, Loss: 1.8535
訓練次數: 1000, Loss: 1.9047
訓練次數: 1100, Loss: 1.9666
訓練次數: 1200, Loss: 1.6923
訓練次數: 1300, Loss: 1.6485
訓練次數: 1400, Loss: 1.73
訓練次數: 1500, Loss: 1.824
訓練花費時間:14.9578
整體測試集上的loss: 305.16554152965546
整體測試集上的正確率: 0.3138
模型已保存
---------第 3 輪訓練開始---------
訓練次數: 1600, Loss: 1.7588
訓練次數: 1700, Loss: 1.6347
訓練次數: 1800, Loss: 1.9415
訓練次數: 1900, Loss: 1.6931
訓練次數: 2000, Loss: 1.8994
訓練次數: 2100, Loss: 1.516
訓練次數: 2200, Loss: 1.4472
訓練次數: 2300, Loss: 1.7994
訓練花費時間:22.914
整體測試集上的loss: 267.35631120204926
整體測試集上的正確率: 0.3819
模型已保存
---------第 4 輪訓練開始---------
訓練次數: